# Импорты

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
import random
import datetime
from datetime import datetime
import warnings
import os, fnmatch
import yadisk
import shutil
import psycopg2
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 
import seaborn as sns
from tqdm import tqdm

warnings.simplefilter('ignore')
options = webdriver.ChromeOptions()
now = datetime.now()
options.headless = True

token = '*****'
dir_ = "*****"

EXE_PATH = r'C:\chromedriver.exe'

dates = {
    'января':'01',
    'февраля':'02',
    'марта':'03',
    'апреля':'04',
    'мая':'05',
    'июня':'06',
    'июля':'07',
    'августа':'08',
    'сентября':'09',
    'октября':'10',
    'ноября':'11',
    'декабря':'12'
}

host_2 = '*****'
user_2 = '*****'
password_2 = '*****'
db_name_2 = '*****'
port_2 = 5432

connection_2 = psycopg2.connect(
host=host_2,
    user=user_2,
    password=password_2,
    database = db_name_2,
    port=port_2
)


connection_2.autocommit = True

SQL_insert = "INSERT INTO commerce_ishevsk (type, adres, area, square, floor, build_type, furnish,\
price, m_price, date, url_link, ad_number, description)\
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"


db_config = {'user': '*****', # имя пользователя
'pwd': '*****', # пароль
'host': '*****',
'port': 5432, # порт подключения
'db': '*****'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string)

y = yadisk.YaDisk(token=token)
print(y.check_token())

True


# Функции

In [16]:
def get_date_type(date):
    try:
        return datetime.strptime(date, "%d.%m.%Y")
    except:        
        return datetime.strptime(date[:10], "%Y-%m-%d")

def get_pages_number():
    pages = driver.find_elements("class name", "pagination-item-JJq_j")
    pages_ = []
    for page in pages:    
        if page.text.isdigit():        
            pages_.append(int(page.text))
    return(max(pages_))

def get_norm_links():
    all_links = []
    elems = driver.find_elements("xpath" ,"//a[@href]")
    for elem in elems:
        all_links.append(elem.get_attribute("href"))
    norm_links = []
    for link in all_links:
        if link.split('_')[-1].isdigit():
            norm_links.append(link)
    return(list(set(norm_links)))

def yandex():
    print('Начинаем перенос скринов на Я.диск')
    # Собираем файлы
    files = []
    listOfFiles = os.listdir('screenshots')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    # переносим в другую директорию
    for file in files:
        shutil.move('screenshots' +'\\' + file, os.getcwd())
    #записываем на Я.Диск
    c = 0        
    for file in files:
        print(f'Записываем {file}')
        if y.is_file(f'{dir_}{file}'):
            print('Такой уже есть')
            continue
        else:
            c+=1
            y.upload(file, f'{dir_}{file}')
    #удаляем файлы       
    files = []
    listOfFiles = os.listdir('.')  
    pattern = "*.png"  
    for entry in listOfFiles:  
        if fnmatch.fnmatch(entry, pattern):            
                files.append(entry)
    for file in files:
        os.remove(file)

    print(f'{c} файлов записано на Я.Диск')

def get_type():
    try:
        try:
            test = driver.find_element("class name", "style-item-navigation-3dyI6")
            return test.text.split('·')[-1]
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-navigation-In5Jr")
            return test.text.split('·')[-1]
        except:
            pass
    except:
        return None

def get_adress():
    try:
        try:
            adress = driver.find_element("class name", "style-item-address__string-3Ct0s")
            return(adress.text)
        except:
            pass
        try:
            adress = driver.find_element("class name", "style-item-address__string-wt61A")
            return(adress.text)
        except:
            pass
    except:       
        return(None)

def get_rayon():
    try:
       
        try:
            rayon = driver.find_element("class name", "style-item-address-georeferences-item-18pFt")
            return(rayon.text)
        except:
            pass
        
        try:
            rayon = driver.find_element("class name", "style-item-address-georeferences-item-TZsrp")
            return(rayon.text)
        except:
            pass
    
    except:        
        return(None)

def get_params():
    try:
        if len(driver.find_elements("class name", "params-paramsList__item-1Xeok")) > 0:
            params = driver.find_elements("class name", "params-paramsList__item-1Xeok")
            params_ = []
            for param in params:
                params_.append(param.text.split(':'))
            param_list = []
            for param in params_:
                  param_list.extend(param)
            return param_list
        
        if len(driver.find_elements("class name", "params-paramsList-zLpAu")) > 0:
            param_list = driver.find_elements("class name", "params-paramsList-zLpAu")[0].text.replace('\n', ' ').split(' ')
            return param_list
    except:        
        return None
    
def get_house_params():
    try:
        if len(driver.find_elements("class name", "style-item-params-list-3YJu7")) > 0:
            house_params = driver.find_elements("class name", "style-item-params-list-3YJu7")
            house_params_ = []

            for param in house_params:
                house_params_.append(param.text.replace('\n',' '))
            if len(house_params_) > 0:
                return(house_params_[0].split(' '))
            else:
                return('ERROR')
        if len(driver.find_elements("class name", "style-item-params-McqZq")) > 0:
            return driver.find_elements("class name", "style-item-params-McqZq")[0].text.replace('\n', ' ').split(' ')[2:]
    except:        
        return None
    
def get_square(kv_par):
    try:
        try:
            x = float(kv_par[kv_par.index('Общая площадь')+1].split()[0])
            return(x)
        except:
            pass
        try:
            x = float(kv_par[kv_par.index('площадь:')+1])
            return(x)
        except:
            pass
    except:        
        return(None)
    
def get_house_type(house_par): 
    try:
        try:
            x = house_par[house_par.index('здания:')+1]
            return(x)
        except:
            print('Ошибка в типе дома')
            print(url)
            return(None)
    except:       
        return(None)
    
def get_kv_floor(kv_par):
    try:
        try:
            x = kv_par[kv_par.index('Этаж:')+1].strip()
            return(x)
        except:
            pass
        try:
            x = kv_par[kv_par.index('Этаж')+1].strip()
            return(x.split()[0].strip())
        except:
            pass
    except:       
        return(None)
    
def get_otdela(kv_par):
    try:
        try:
            x = kv_par[kv_par.index('Отделка')+1].strip()
            return(x)        
        except:       
            pass
        try:
            x = kv_par[kv_par.index('Отделка:')+1].strip()
            return(x)  
        except:
            pass
    except:
        return None
    
def get_price():
    try:
        test = driver.find_element("class name", "style-item-view-price-content-1aHxn")
    except:
        pass
    try:
        test = driver.find_element("class name", "style-item-price-PuQ0I")
    except:
        pass
    i = 0
    price = ''
    while True:
        if test.text[i].isdigit() or test.text[i] == ' ':
            price += test.text[i]
            i +=1
        elif i > len(test.text):
            break
        else:
            break
    try:
        return(float(price.replace(' ', '')))
    except:        
        return(None)

def get_date():
    try:
        #1 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-1y5w6")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        # 2 ексепт
        try:
            test = driver.find_element("class name", "style-item-metadata-date-EEthc")
            return(' '.join(test.text.split()[:2]) + ' ' + str(time.localtime().tm_year))
        except:
            pass
        
        #3 ексепт
        try:
            test = driver.find_element("class name", "style-item-footer-Ufxh_")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
        
        try:
            test = driver.find_element("class name", "style-item-footer-3uXQz")
            if 'сегодн' in test.text.split()[3]:
                return(str(now.day) +'.'+  str(now.month)+'.'+ str(now.year))
            if 'вчер' in test.text.split()[3]:                
                return(str(now.day-1) +'.'+  str(now.month)+'.'+ str(now.year))
            else:
                date = (test.text.split()[3]+ ' '+ test.text.split()[4]+' '+ str(now.year))
                date = date.split()
                date[1] = dates[date[1]]
                return('.'.join(date))
        except:
            pass
        
        
    except:        
        return(None)
    
def is_bad_adv(i,j):
    try:
        driver.find_element("class name", "desktop-16c9mq7").text
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        print(url)
        print('Страницы не существует')
        return(True)
    except:        
        try:
            driver.find_element("class name", "closed-warning-content-2ooy4").text
            print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
            print('Снято с публикации')
            print(url)
            return(True)
        except:
            try:
                driver.find_element("class name", "page-title-count-wQ7pG").text
                print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                print(url)
                print('Страница поиска, объявления нет')
                return(True)
            except:
                    try:
                        driver.find_element("class name", "b-404").text
                        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
                        print('Нет такой страницы')
                        print(url)
                        return(True)
                    except:
                        return(False)
def get_view():
    try:
        try:
            test = driver.find_element("class name", "style-item-description-html-qCwUL")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-mc3G6")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-html-1_RNo")
            return(test.text.replace('\n', ' '))
        except:
            pass
        try:
            test = driver.find_element("class name", "style-item-description-text-SzN56")
            return(test.text.replace('\n', ' '))
        except:
            pass
    except:        
        return(None)

# Links

In [17]:
# количество страниц
driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
driver.get('https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/prodam-ASgBAgICAUSwCNJW?f=ASgBAgICAkSwCNJW8hKg2gE&p=1')
page_numbers = get_pages_number()
driver.close()
print(f'Количество страниц с ссылками: {page_numbers}')

links = []
#Парсим линки
for i in tqdm(range(1, page_numbers+1)):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(f'https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/prodam-ASgBAgICAUSwCNJW?f=ASgBAgICAkSwCNJW8hKg2gE&p={i}')
    links.extend(get_norm_links())
    driver.close()
    time.sleep(random.randint(5,12))

links = set(links)

with connection_2.cursor() as cursor:
                    cursor.execute(
                    'TRUNCATE TABLE commerce_links'
                    )

query = "SELECT * FROM commerce_ishevsk"
old_links = list(pd.io.sql.read_sql(query, con = engine)['url_link'].values)         
        
for link in links:
    if 'izhevsk' in link and link not in old_links:
        with connection_2.cursor() as cursor:
                    cursor.execute(
                    f'''INSERT INTO commerce_links (links) VALUES ('{link}')'''
                    )
    else:
        continue
        
query = "SELECT links FROM commerce_links"
print(f'Новых линков: {len(list(pd.io.sql.read_sql(query, con = engine)["links"].values))}')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

Количество страниц с ссылками: 12


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [07:00<00:00, 35.04s/it]


Новых линков: 40


# Парсинг

In [18]:
#Обнуляем счетчик
counts_ = '0'

with connection_2.cursor() as cursor:
    cursor.execute(
    f'''UPDATE commerce_count
    SET count = {counts_}'''
    )

In [19]:
query = "SELECT * FROM commerce_count"
test = pd.io.sql.read_sql(query, con = engine) 
i = int(test['count'].values[0])
i

0

## Сам парсер

In [20]:
query = "SELECT * FROM commerce_links"
test = pd.io.sql.read_sql(query, con = engine) 
links = list(test['links'].values)

query = "SELECT * FROM commerce_count"
test = pd.io.sql.read_sql(query, con = engine) 
i = int(test['count'].values[0])
# Пустой ДФ
options.headless = True
df = {}
df['Тип помещения'] = [] 
df['Адрес'] = [] 
df['Район'] = []  
df['Общая площадь'] = [] 
df['Этаж'] = [] 
df['Тип здания'] = []
df['Отделка'] = []
df['Стоимость'] = [] 
df['Метр квадратный'] = [] 
df['Дата объявления'] = []
df['Ссылка'] = [] 
df['номер объявления'] = [] 
df['Описание'] = [] 

for j,url in enumerate (links[i:]):
    driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
    driver.get(url)
    
    
    if j % 5 == 0 and j > 1:
        counts_ = str(i+j)
        
        with connection_2.cursor() as cursor:
            cursor.execute(
            f'''UPDATE commerce_count
            SET count = {counts_}'''
            )
        
        
        df = pd.DataFrame(df)
        for q in range (len(df)):
            values_ = []
            for value in df.loc[q].values:
                values_.append(str(value))
            with connection_2.cursor() as cursor:
                    cursor.execute(
                    SQL_insert, [*values_]
                    )
        df = {}
        df['Тип помещения'] = [] 
        df['Адрес'] = [] 
        df['Район'] = []  
        df['Общая площадь'] = [] 
        df['Этаж'] = [] 
        df['Тип здания'] = []
        df['Отделка'] = []
        df['Стоимость'] = [] 
        df['Метр квадратный'] = [] 
        df['Дата объявления'] = []
        df['Ссылка'] = [] 
        df['номер объявления'] = [] 
        df['Описание'] = []
        print(f'Данные записаны на {i+j} позиции')
        
        yandex()
        
    if is_bad_adv(i,j): 
        driver.close()
        time.sleep(2)
        continue    
    try:            
        kv_par = get_params()
        house_par = get_house_params()
        #print('---'*22)
        df['Тип помещения'].append(get_type())
        df['Адрес'].append(get_adress())
        df['Район'].append(get_rayon())
        df['Общая площадь'].append(get_square(kv_par))
        df['Этаж'].append(get_kv_floor(kv_par))
        df['Тип здания'].append(get_house_type(house_par))
        df['Отделка'].append(get_otdela(kv_par))
        df['Стоимость'].append(get_price())
        
        df['Дата объявления'].append(get_date())
        df['Ссылка'].append(url)
        df['номер объявления'].append(df['Ссылка'][-1].split('_')[-1])
        df['Описание'].append(get_view())
        try:
            df['Метр квадратный'].append(round(df['Стоимость'][-1]/df['Общая площадь'][-1],2))
        except:
            print('Ошибка в определении стоимости метра кв.')
            df['Метр квадратный'].append(None)
                
        print(f'{"***"*14} № {(j+i)} из {len(links)} {"***"*14}')
        if df["Адрес"][-1] == None:
            print(f'Адрес: {df["Адрес"][-1]}')
        if df["Район"][-1] == None:
            print(f'Район: {df["Район"][-1]}')
        if df["Общая площадь"][-1] == None:
            print(f'Общая площадь: {df["Общая площадь"][-1]}')
        if df["Этаж"][-1] == None:
            print(f'Этаж: {df["Этаж"][-1]}')        
        if df["Стоимость"][-1] == None:
            print(f'Стоимость: {df["Стоимость"][-1]}')
        if df["Дата объявления"][-1] == None:
            print(f'Дата объявления: {df["Дата объявления"][-1]}')
        print(f'Ссылка: {df["Ссылка"][-1]}')
        if df["Отделка"][-1] == None:
            print(f'Отделка: {df["Отделка"][-1]}')
        if df["Тип здания"][-1] == None:
            print(f'Тип здания: {df["Тип здания"][-1]}')
        if df["Описание"][-1] == None:
            print(f'Описание: {df["Описание"][-1]}')
                        
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element('tag name' ,'body').screenshot(f'screenshots/{url.split("_")[-1]}.png')
        print('Скриншот сделан')
        driver.quit()
        
        print()
        
        #print('---'*22)
        

    except:
        print('Тут что-то пошло не так (ошибка с вылетом скрипта):')
        print(url)        
        break
        
        #j +=1
    
    time.sleep(random.randint(10,19))

else:
    df = pd.DataFrame(df)
    for q in range (len(df)):
        values_ = []
        for value in df.loc[q].values:
            values_.append(str(value))
        with connection_2.cursor() as cursor:
                cursor.execute(
                SQL_insert, [*values_]
                )
    yandex()
    print('###'*22)
    print('#'+' '*64+'#')
    print('#'+ ' ' *20 + 'СКРИПТ ОКОНЧИЛ РАБОТУ!' + ' '*22 + '#')
    print('#'+' '*64+'#')
    print('###'*22)

****************************************** № 0 из 40 ******************************************
Этаж: None
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/gipermarket_izhtreyding_947_kv_m_g._izhevsk_ul.len_2249938009
Скриншот сделан

****************************************** № 1 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_naznacheniya_54_m_2422834320
Скриншот сделан

****************************************** № 2 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/pomeschenie_svobodnogo_naznacheniya_324.9_m_1955317057
Скриншот сделан

****************************************** № 3 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/prodaetsya_pomeschenie_s_arendatorom_256.5_m_2490524451
Скриншот сделан

****************************************** № 4 из 40 ***********

Такой уже есть
Записываем 2319812400.png
Записываем 2411247939.png
Такой уже есть
Записываем 2543000542.png
Записываем 2560671360.png
3 файлов записано на Я.Диск
****************************************** № 30 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/prodam_proizvodstvennoe_pomeschenie_551_m_2401283415
Отделка: None
Скриншот сделан

****************************************** № 31 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_naznacheniya_84_m_2572797700
Скриншот сделан

****************************************** № 32 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_nedvizhimost/svobodnogo_naznacheniya_62.8_m_2555832770
Скриншот сделан

****************************************** № 33 из 40 ******************************************
Ссылка: https://www.avito.ru/izhevsk/kommercheskaya_ned

# Предобработка

In [21]:
query = "SELECT * FROM commerce_ishevsk"
df = pd.io.sql.read_sql(query, con = engine)
df = df[df.columns[1:]]
df.head()

,type,adres,area,square,floor,build_type,furnish,price,m_price,date,url_link,ad_number,description
0,Торговое помещение,"Удмуртская Республика, Ижевск, ул. Ворошилова,...",р-н Устиновский,1442.5,1,бизнес-центр,чистовая,39542485.0,27412.47,19.08.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2479793562,"Торгово-офисные помещения в бизнес-центре ""Дом..."
1,Производственное помещение,"Удмуртская Республика, Ижевск, ул. Гагарина, 104",р-н Ленинский,9014.1,3,другой,чистовая,49696939.0,5513.24,8.09.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2284297331,"Продается комплекс зданий, производственно-скл..."
2,Помещение свободного назначения,"Удмуртская Республика, Ижевск, ул. Пастухова, 43",р-н Первомайский,86.0,1,жилой,офисная,6550000.0,76162.79,15.08.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2117676983,"Продам помещение в центре , на перекрёстке Пуш..."
3,Торговое помещение,"Удмуртская Республика, Ижевск, Июльская ул., 20",р-н Ленинский,47.0,цокольный,торговый,чистовая,1300000.0,27659.57,5.09.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2570212556,"Продается готовый бизнес ""Отдел сантехники, кр..."
4,Помещение свободного назначения,"Удмуртская Республика, Ижевск, ул. имени Татья...",р-н Устиновский,62.0,1,жилой,чистовая,2390000.0,38548.39,25.08.2022,https://www.avito.ru/izhevsk/kommercheskaya_ne...,2116797867,Продам помещение свободного назначения в Устин...


In [22]:
df.duplicated(subset=['ad_number']).sum()

11

In [23]:
df = df.drop_duplicates(subset=['ad_number'])
cols = df.columns
for col in cols:
    print(f'{col}: {df[col].isna().sum()}')

type: 0
adres: 0
area: 0
square: 0
floor: 0
build_type: 0
furnish: 0
price: 0
m_price: 0
date: 0
url_link: 0
ad_number: 0
description: 0


In [26]:
for col in cols:
    print(f'{col}: {(df[col] == "nan").sum()}')

type: 0
adres: 0
area: 0
square: 0
floor: 0
build_type: 0
furnish: 0
price: 0
m_price: 0
date: 0
url_link: 0
ad_number: 0
description: 0


In [25]:
df = df[df['square'] != 'nan']

In [29]:
for col in cols:
    print(f'{col}: {(df[col] == "None").sum()}')

type: 0
adres: 0
area: 27
square: 0
floor: 51
build_type: 0
furnish: 37
price: 0
m_price: 0
date: 0
url_link: 0
ad_number: 0
description: 0


In [28]:
df = df[df['build_type'] != 'None']

In [30]:
df['square'] = df['square'].apply(float)
df['price'] = df['price'].apply(float)
df['m_price'] = df['m_price'].apply(float)
df['datetime'] = df['date'].apply(get_date_type)

In [31]:
df.to_excel('commerce_izhevsk.xlsx', index=0)